# Detecção de Regiões Verdes Em Imagens de Satélites
## Proposta Utilizando Python e OpenCV

In [2]:
# importacao de bilbiotecas utilizadas
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [3]:
def decorrstretch(A):
    A = A.reshape([-1,3]).astype(np.float)
    
    cov = np.cov(A.T)
    sigma = np.diag(np.sqrt(cov.diagonal()))
    eigval, V = np.linalg.eig(cov)
    S = np.diag(1/np.sqrt(eigval))
    mean = np.mean(A, axis=0)
    A -= mean
    T = np.dot(np.dot(np.dot(sigma, V), S), V.T)
    #T = np.dot(np.dot(V.T, S), V)
    offset = mean - np.dot(mean, T)    
    A = np.dot(A, T)
    A += mean + offset
    B = A.reshape([600,1000,3])
    for b in range(3):
        B[:,:,b] = 255 * (B[:,:,b] - B[:,:,b].min())/(B[:,:,b].max() - B[:,:,b].min())
        
    return B.astype(np.uint8)

In [4]:
# listamos os arquivos no diretorio "satelite"
files = os.listdir("satelite/")

# itera sobre as imagens classificando a zona
for file in files:
    # leitura das imagens usando OpenCV
    img_old = cv2.imread("satelite/" + file)
    figname = file[:len(file) - 4]
    
    # suavizacao por filtro Gaussiano
    img_old = cv2.GaussianBlur(img_old, (37, 37), 0)
    cv2.imwrite("modificado/" + figname + "_blur.png", img_old)

    # espalhamento do espectro de cores
    MIN = img_old.min()
    MAX = img_old.max()
    for i in range(3):
        img_old[:,:,i] = (255 / (MAX - MIN)) * (img_old[:,:,i] - MIN)
    cv2.imwrite("modificado/" + figname + "_equalize.png", img_old)

    # aumentamos constraste de cores
    # decorrelacao por alongamento
    img_old = decorrstretch(img_old)
    cv2.imwrite("modificado/" + figname + "_decorr.png", img_old)
    
    # avaliamos diferentes limiares de verde
    for c in range(50, 0, -5):
        # definimos as areas verdes usando
        # uma avaliaca sobre o espaco YUV        
        img = img_old.copy()
        for i in range(600):
            for j in range(1000):
                r,g,b = img[i][j][:]

                y = 0.299 * r + 0.587 * g + 0.114 * b
                u = 0.436 * (b - y) / (1 - 0.114)
                v = 0.615 * (r - y) / (1 - 0.299)
                
                img[i][j][1] = 255 if (u < -c and y < 150) else 0
        cv2.imwrite("modificado/" + figname + "_u" + str(c) +".png", img)
    
    # mascara ótima por inspeção c = 22 y < 150
    img = img_old.copy()
    for i in range(600):
        for j in range(1000):
            r,g,b = img[i][j][:]
            
            y = 0.299 * r + 0.587 * g + 0.114 * b
            u = 0.436 * (b - y) / (1 - 0.114)
            v = 0.615 * (r - y) / (1 - 0.299)
            
            # mascara final zerando canais R e B
            # para pixels fora do limiar desejado
            img[i][j][0] = 0
            img[i][j][2] = 0
            img[i][j][1] = 255 if (u < -22 and y < 150) else 0
    # aplicamos um filtro para eliminação de ruídos
    # e regioes pouco representativas (pequenas)
    img = cv2.medianBlur(img, 35)
    cv2.imwrite("modificado/" + figname + "_mask.png", img)
        
    print(file)

MG araxa.png
MG capetinga.png
MG mirai.png
MG passos.png
MG tres_coracoes.png
PB joao_pessoa.png
PR roncador.png
RN diogo_lopes.png
RN natal.png
RS caxias_do_sul.png
SP cerquilho.png
SP santa_rita_do_passaquatro.png
SP sao_carlos.png
